In [85]:
import pandas as pd
import os
import numpy as np

In [86]:
if os.name == 'nt':
    sep = '\\'
elif os.name == 'posix':
    sep = '/'
else:
    print(f'What is this OS? {os.name}')

path = os.getcwd()
path_datasets = path[:-len(f'Code{sep}src{sep}project_CSP_MATH_571')] + f'DataSets{sep}'

# Read csv
df = pd.read_csv(path_datasets + 'Holidays_data.csv', delimiter=',')
#display(df)

In [87]:
df = df[['date', 'daytype']].drop_duplicates()

In [88]:
print('number of days:', len(df))
print('number of expected approximate days:', 365*20)
print('Percentage of missing days:', 100 - (len(df) / (365.25 *20)) * 100)

number of days: 6939
number of expected approximate days: 7300
Percentage of missing days: 5.010266940451743


### As we don't have all the days in the dataset, several approaches could be taken:
        - Check if the time period we need has missing values
        - Find another dataset with no missing values
        - Fill some of the missing values

In [110]:
df_2019 = df[df['date'].str.contains('2019')]
df_2018 = df[df['date'].str.contains('2018')]
df_2017 = df[df['date'].str.contains('2017')]
df_2016 = df[df['date'].str.contains('2016')]

print('Ammount of days in 2019:', len(df_2019))
print('Ammount of days in 2018:', len(df_2018))
print('Ammount of days in 2017:', len(df_2017))
print('Ammount of days in 2016:', len(df_2016))
print()
print('Number of missing values in daytype for 2019:',np.sum(df_2019.isnull()['daytype']) )
print('Number of missing values in daytype for 2018:',np.sum(df_2018.isnull()['daytype']) )
print('Number of missing values in daytype for 2017:',np.sum(df_2017.isnull()['daytype']) )

df_2016_2019 = df[df['date'].str.contains('2019|2018|2017|2016')]
assert(len(df_2016_2019) == ((365*3)+366))

Ammount of days in 2019: 365
Ammount of days in 2018: 365
Ammount of days in 2017: 365
Ammount of days in 2016: 366

Number of missing values in daytype for 2019: 0
Number of missing values in daytype for 2018: 0
Number of missing values in daytype for 2017: 0


### Therefore it seems the last years (the ones we need) contain all the days

In [90]:
df_taxi = pd.read_csv(path_datasets + 'jan-ap19.csv', delimiter=',')

In [91]:
df_taxi['trip_start_timestamp']
# As it can be seen, it has to be divided into only day
# Then, the format has to be changed

0         2019-01-17 11:30:00 UTC
1         2019-03-02 09:45:00 UTC
2         2019-02-12 11:15:00 UTC
3         2019-02-15 03:30:00 UTC
4         2019-02-27 07:00:00 UTC
                   ...           
799255    2019-03-13 16:45:00 UTC
799256    2019-03-13 16:45:00 UTC
799257    2019-03-15 07:45:00 UTC
799258    2019-03-11 15:15:00 UTC
799259    2019-03-11 15:15:00 UTC
Name: trip_start_timestamp, Length: 799260, dtype: object

In [92]:
df_date = df_taxi['trip_start_timestamp'].str.split(' ').apply(lambda x : x[0])

In [103]:
dates = pd.to_datetime(df_date)
df_taxi['date'] = dates.dt.strftime('%m/%d/%Y')

In [111]:
df_taxi_merged = df_taxi.merge(df_2016_2019, how = 'left', on = 'date')

In [112]:
len(df_taxi_merged) == len(df_taxi)

True

In [113]:
df_taxi_merged

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location,date,daytype
0,ececa29f7a1392f7d937f34a2040f409744d0354,8d1222551aa0783ed24c0941a29222b21fe66e643a801a...,2019-01-17 11:30:00 UTC,2019-01-17 12:00:00 UTC,1363.0,15.50,NaN,NaN,41.0,NaN,...,Cash,Flash Cab,41.794090,-87.592311,POINT (-87.592310855 41.794090253),NaN,NaN,NaN,01/17/2019,W
1,1fbf17a48aca428fefc0a79d0218ed729ea7e8e3,c0efb2f0d92d8721d64fcd6628a9f8e78b7693cb5f36ee...,2019-03-02 09:45:00 UTC,2019-03-02 10:45:00 UTC,2754.0,13.73,NaN,NaN,46.0,NaN,...,Cash,Flash Cab,41.741243,-87.551428,POINT (-87.551428197 41.7412427285),NaN,NaN,NaN,03/02/2019,A
2,73eab95221395356ad45ce8b7a8716bcddb4c034,5e00ac77728ae1790a517495300680a2ff69fe9a569218...,2019-02-12 11:15:00 UTC,2019-02-12 12:15:00 UTC,3422.0,31.77,NaN,NaN,8.0,NaN,...,Cash,Flash Cab,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),NaN,NaN,NaN,02/12/2019,W
3,701fcdd02b0dd0dc01cbffcd7c2c9572386f5966,e5e1bb9c3329c0f9bd1f291cb9bbbb016731c148fefca8...,2019-02-15 03:30:00 UTC,2019-02-15 03:45:00 UTC,1302.0,10.16,NaN,NaN,8.0,NaN,...,Cash,Flash Cab,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),NaN,NaN,NaN,02/15/2019,W
4,2984246ff88a9e42cb7d80d5a056a91592cd2103,0574d247700e50d1fb996084c8b1c649bf57effe419a6d...,2019-02-27 07:00:00 UTC,2019-02-27 07:45:00 UTC,2308.0,14.26,NaN,NaN,8.0,NaN,...,Cash,Flash Cab,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),NaN,NaN,NaN,02/27/2019,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799255,b1fd61b2b701b0d00d4de31b025a3325b0b5731d,c8d54c218b94d091de91c543a7a9884778bed1c4070106...,2019-03-13 16:45:00 UTC,2019-03-13 18:00:00 UTC,4391.0,15.46,1.703106e+10,1.703198e+10,6.0,76.0,...,Cash,Flash Cab,41.938391,-87.638575,POINT (-87.6385749205 41.938391257700005),41.979071,-87.90304,POINT (-87.9030396611 41.9790708201),03/13/2019,W
799256,73f33b2d9761825b5ccc00cac02b49927fba63ea,c8d54c218b94d091de91c543a7a9884778bed1c4070106...,2019-03-13 16:45:00 UTC,2019-03-13 18:00:00 UTC,4262.0,15.45,1.703106e+10,1.703198e+10,6.0,76.0,...,Cash,Flash Cab,41.938391,-87.638575,POINT (-87.6385749205 41.938391257700005),41.979071,-87.90304,POINT (-87.9030396611 41.9790708201),03/13/2019,W
799257,373ba76eb995f485cc339993ca962e4a9a382476,4628ef9dfa973bdfe877c5aa9d9738f9dc1204e54f2f1a...,2019-03-15 07:45:00 UTC,2019-03-15 08:45:00 UTC,3119.0,15.99,1.703106e+10,1.703198e+10,6.0,76.0,...,Credit Card,Flash Cab,41.938391,-87.638575,POINT (-87.6385749205 41.938391257700005),41.979071,-87.90304,POINT (-87.9030396611 41.9790708201),03/15/2019,W
799258,b3d17e9958689003f8522b2257fe76419d1c07f4,def4d7f708bad8d98ae110c6986f6b77558d0f431abe51...,2019-03-11 15:15:00 UTC,2019-03-11 16:00:00 UTC,2799.0,15.65,1.703106e+10,1.703198e+10,6.0,76.0,...,Cash,Flash Cab,41.938391,-87.638575,POINT (-87.6385749205 41.938391257700005),41.979071,-87.90304,POINT (-87.9030396611 41.9790708201),03/11/2019,W
